In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
training = pd.read_csv("/kaggle/input/titanic/train.csv")


In [5]:
test = pd.read_csv("/kaggle/input/titanic/test.csv")


In [6]:
train_data = training.copy()
train_data.set_index('PassengerId', inplace = True, drop = True)
print('Shape: ', train_data.shape)
print(train_data.info())

In [7]:
test_data = test.copy()
test_data.set_index('PassengerId', inplace=True, drop=True)
print('Shape : ',test_data.shape)
print(test_data.info())

In [9]:
age_test_data = test_data.copy()
age_train_data = train_data.copy()

## Ajustes de edades

Creamos una lista de con las formulas de tratamiento obtenidas de los nombres de pasajeros

In [10]:
def Create_salutation(df):
    Split_name1 = df['Name'].str.split(',', expand=True)
    Split_name2 = Split_name1[1].str.split('.', expand=True)
    Split_name2[0].unique()
    Split_name2[0]=Split_name2[0].str.replace(' ', '')
    df['salutation'] = Split_name2[0]
    
Create_salutation(age_test_data)
Create_salutation(age_train_data)

In [11]:
print('Train Data : ',age_train_data[age_train_data['Age'].isnull()]['salutation'].unique())
print('Data : ',age_test_data[age_test_data['Age'].isnull()]['salutation'].unique())

In [12]:
train_sample = age_train_data[age_train_data['salutation'].isin(['Mr', 'Mrs', 'Miss', 'Master', 'Dr'])]
train_age_summary = train_sample.groupby(['salutation', 'Pclass'])['Age'].agg(['mean', 'count'])
train_age_summary

In [13]:
list(train_age_summary['mean'])

In [14]:
age_train_data.info()

In [15]:
salutations = ['Dr','Master','Miss','Mr','Mrs']
pclasses = [1,2,3]
ages = [43.75,
 38.5,
 0,
 5.306666666666667,
 2.2588888888888885,
 5.350833333333333,
 30.0,
 22.390625,
 16.1231884057971,
 41.58045977011494,
 32.76829268292683,
 28.724890829694324,
 40.88235294117647,
 33.68292682926829,
 33.515151515151516]
for stn in range(len(salutations)):
    for pcl in range(len(pclasses)):
        age = (stn*3)+pcl
        age_train_data.loc[(age_train_data['Age'].isnull()) & (age_train_data['salutation']==salutations[stn]) & (age_train_data['Pclass'] == pclasses[pcl]),'Age'] = ages[age] 

In [16]:
age_train_data.info()

In [17]:
age_test_data[age_test_data['Age'].isnull()]['salutation'].unique()

In [18]:
train_sample = age_train_data[age_train_data['salutation'].isin(['Mr', 'Mrs', 'Miss', 'Master', 'Dr'])]
train_age_summary = train_sample.groupby(['salutation', 'Pclass'])['Age'].agg(['mean', 'count'])
train_age_summary

In [19]:
list(train_age_summary['mean'])

In [20]:
age_train_data.info()

In [21]:
age_test_data[age_test_data['Age'].isnull()]['salutation'].unique()

In [22]:
test_sample = age_test_data[age_test_data['salutation'].isin(['Mr', 'Mrs', 'Miss', 'Ms', 'Master'])]
test_age_summary = test_sample.groupby(['salutation', 'Pclass'])['Age'].agg(['mean', 'count'])
test_age_summary

In [23]:
list(test_age_summary['mean'])

In [24]:
ages_test = [9.5,
 5.0,
 7.454615384615384,
 31.428571428571427,
 17.37,
 19.872647058823528,
 41.2,
 31.71818181818182,
 27.198795180722893,
 45.60606060606061,
 33.0,
 29.875,
 29.875,
 29.875,
 29.875,]
salutations_test = ['Mr', 'Mrs', 'Miss', 'Ms', 'Master']
pclasses = [1,2,3]
for stn in range(len(salutations_test)):
    for pcl in range(len(pclasses)):
        age = (stn*3)+pcl
        age_test_data.loc[(age_test_data['Age'].isnull()) & (age_test_data['salutation']==salutations_test[stn]) & (age_test_data['Pclass'] == pclasses[pcl]),'Age'] = ages_test[age]        
        

In [25]:
age_test_data.info()

In [26]:
train_data = age_train_data
test_data = age_test_data

Eliminamos columnas no necesarias Name, ticket, Fare, Cabin, Embarked y Salutation


In [27]:
#Eliminamos columnas no necesarias Name, ticket, Fare, Cabin, Embarked y Salutation
train_data.columns.values

In [28]:
drop_list = ['Name', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'salutation']
train_data = train_data.drop(drop_list, axis=1)
test_data = test_data.drop(drop_list,axis=1)

In [29]:
#Convertimos genero Male y Female a 0 y 1
train_data['Sex'] = train_data['Sex'].map({'male':0, 'female':1})
test_data['Sex'] = test_data['Sex'].map({'male':0, 'female':1})

In [30]:
test_data

In [31]:
train_data.describe()

In [32]:
features = ['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch']
plt.figure(figsize=(20,20))

for ftr in range(len(features)):
    plt.subplot(2,3,ftr+1)
    plt.hist([train_data[train_data['Survived']==0][features[ftr]],train_data[train_data['Survived']==1][features[ftr]]],bins=8, label=["Dead", "Survive"])
    plt.legend()
    plt.xlabel(features[ftr])
    plt.ylabel('Count')



plt.show()

In [33]:
sns.pairplot(train_data)

Scale y transformación de datos

In [36]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [37]:
X = train_data.drop('Survived',axis=1)
Y = train_data['Survived']

In [38]:
X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size = 0.2, random_state = 42 )
x_train, x_test = X_train, X_test

In [39]:

scaler = StandardScaler()
scaler.fit(X)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [40]:
from sklearn.ensemble import RandomForestClassifier

In [41]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, y_train)
acc_random_forest = round(random_forest.score(X_train, y_train) * 100, 2)
acc_random_forest

In [42]:
submissionDF = test_data.copy()
submissionDF = submissionDF.drop(['Pclass','Sex','Age','SibSp','Parch'],axis=1)
testData = scaler.transform(test_data)

y_pred_final = random_forest.predict(testData)


submissionDF['Survived'] = y_pred_final
submissionDF.to_csv('submission.csv')